In [1]:
import os
import numpy as np

from bisect import bisect_right
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [10]:


class LibriSpeechDataset(Dataset):
    
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.npzs = [fn for fn in os.listdir(data_dir) if fn[-4:] == '.npz']
        self.bookmark = [0]
                
        bookmark_cnt = 0

        for npz in tqdm(self.npzs):
            word_cnt = np.load(os.path.join(data_dir, npz))['word_cnt']
            bookmark_cnt += word_cnt
            self.bookmark.append(bookmark_cnt)
    
    def __len__(self):
        return self.bookmark[-1]
    
    def __getitem__(self, idx):
        ni = bisect_right(self.bookmark, idx)    # locate npz file
        npz = self.npzs[ni - 1]
        data = np.load(os.path.join(self.data_dir, npz))
        d = data[f'arr_{idx - self.bookmark[ni - 1]}']
        
        return d


In [20]:
dataset = LibriSpeechDataset('data')
# dataset.idx_bookmark
iter_dataloader = iter(DataLoader(dataset))

data = np.load(f'data/{dataset.npzs[0]}')
iter_num = iter(range(1000))

100%|██████████████████████████████████████████| 87/87 [00:00<00:00, 372.36it/s]


In [19]:
data['arr_0']
next(iter_dataloader)

tensor([[  48,   25,   34,  ..., 1697, 1222, 1070]], dtype=torch.int16)

In [12]:
%time
next(iter_dataloader)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


tensor([[  48,   25,   34,  ..., 1697, 1222, 1070]], dtype=torch.int16)

In [21]:
import torch
i = 0
while i < 100:
    print(i)
    torch.eq(next(iter_dataloader), torch.tensor(data[f'arr_{next(iter_num)}'])).all()
    i += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [24]:
data = np.load(f'data/{dataset.npzs[1]}')
data['arr_1']

array([-9239, -9466, -9471, ...,   237,   291,   249], dtype=int16)

In [25]:
dataset[624]

array([-9239, -9466, -9471, ...,   237,   291,   249], dtype=int16)

In [19]:
data['word_cnt']

array(721)

In [ ]:
# read all the npy files and check sizes
# let's say they already sorted
# index

# binary search on it
# open the file 

In [ ]:
# if memmap it doesn't make any sense to keep opening files
# just open everything once 
# 